In [146]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, util
from gliner import GLiNER
from deep_translator import MyMemoryTranslator
import ast

## Modelo Analisis de sentimientos

In [147]:
dataset_sentimientos = pd.read_csv('./databases/sentimientos.csv')

In [148]:
dataset_sentimientos.head()

,clase,frase
0,0,me siento un poco nostalgico hoy.
1,0,"todo parece tan gris, como si algo faltara."
2,0,"no tengo ganas de hacer nada, estoy algo decaido."
3,0,echo de menos esos tiempos felices que ya no v...
4,0,"me invade una sensacion de tristeza, sin motiv..."


In [149]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1') 

labels = [(0, "Melancolico"), (1, "Ni fu Ni fa"), (2, "Alegre")]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(dataset_sentimientos['frase'].to_list(), dataset_sentimientos['clase'].to_list(), test_size=0.2, shuffle=True,random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)
# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_sentimientos = LogisticRegression(max_iter=100, solver='lbfgs')
modelo_sentimientos.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_sentimientos = modelo_sentimientos.predict(X_test_vectorized)
acc_sentimientos = accuracy_score(y_test, y_pred_sentimientos)
report_sentimientos = classification_report(y_test, y_pred_sentimientos, zero_division=1)

print("Precisión Regresión Logística:", acc_sentimientos)
print("Reporte de clasificación Regresión Logística:\n", report_sentimientos)


Precisión Regresión Logística: 0.9393939393939394
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       0.91      0.91      0.91        11
           2       1.00      1.00      1.00        11

    accuracy                           0.94        33
   macro avg       0.94      0.94      0.94        33
weighted avg       0.94      0.94      0.94        33



In [150]:
for i in range(len(y_test)):
    if y_test[i] != y_pred_sentimientos[i]:
        print(f"Diferencia encontrada en el índice {i}:")
        print(f"X_test: {X_test[i]}")
        print(f"y_test: {y_test[i]}")
        print(f"y_pred_sentimientos: {y_pred_sentimientos[i]}")
        print()  # Línea en blanco para separar las salidas

Diferencia encontrada en el índice 8:
X_test: me siento como si el dia hubiera pasado desapercibido, sin nada memorable.
y_test: 1
y_pred_sentimientos: 0

Diferencia encontrada en el índice 12:
X_test: no encuentro consuelo en nada, todo parece sombrio.
y_test: 0
y_pred_sentimientos: 1



In [151]:
# Make something to export the model to another notebook
import joblib

joblib.dump(modelo_sentimientos, './models/modelo_sentimientos.pkl')

print("Modelo exportado correctamente")

Modelo exportado correctamente


## Modelo Recomendaciones

In [152]:
juegos_dataset = pd.read_csv('./databases/juegos_procesados.csv')
libros_dataset = pd.read_csv('./databases/libros_procesados.csv')
peliculas_dataset = pd.read_csv('./databases/peliculas_procesados.csv')

In [153]:
modelo_recomendaciones = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [163]:
modelo_gliner = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\Usuario\OneDrive\Escritorio\archivos\projects\NLP_Bravi\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
joblib.dump(modelo_recomendaciones, './models/modelo_recomendaciones.pkl')
joblib.dump(modelo_gliner, './models/modelo_gliner.pkl')
print("Modelo exportado correctamente")

## Embeddings

In [ ]:
def traducir(query):
    translated = MyMemoryTranslator(source='spanish', target='english').translate(query)
    return translated

In [164]:
juegos_dataset['embeddings'] = juegos_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))
libros_dataset['embeddings'] = libros_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))
peliculas_dataset['embeddings'] = peliculas_dataset['description'].apply(lambda x: modelo_recomendaciones.encode(x))

In [165]:
def busqueda_peliculas(consulta):
    #convierto los valores de las columnas en listas
    peliculas_dataset['actors'] = peliculas_dataset['actors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    peliculas_dataset['categories'] = peliculas_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Name','Genre','Year']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    names = []
    years = []
    genres = []
    for entidad in entidades_consulta:
        if entidad['label'] == 'Name':
            if entidad['text'].lower() == 'i':
                continue
            names.append(entidad['text'].lower())
        if entidad['label'] == 'Year':
            years.append(entidad['text'].lower())
        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())

    candidate_movies = peliculas_dataset.copy()
    # Filtrar por nombres en 'director' o 'actors'
    if len(names) > 0:
        candidate_movies = candidate_movies = peliculas_dataset[
        peliculas_dataset.apply(lambda row: any(name in [actor for actor in row['actors']] or name in [director for director in row['director']] for name in names), axis=1)]
     # Filtrar por género en 'categories' (solo si hay géneros extraídos)
    if len(genres) > 0:
        candidate_movies = candidate_movies[
            candidate_movies.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]

    # Filtrar por años en 'years' (solo si hay años extraídos)
    if len(years) > 0:
        candidate_movies = candidate_movies[
            candidate_movies.apply(lambda row: any(str(year) in str(row['year']) for year in years), axis=1)
        ]

    # Mostrar el resultado final
    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)

    coseno = util.cos_sim(user_embedding, candidate_movies['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    # Extraer las recomendaciones
    recomendaciones = [candidate_movies['title'].iloc[i] for i in index]
    return recomendaciones


In [166]:
def busqueda_libros(consulta):
    #convierto los valores de las columnas en listas
    libros_dataset['categories'] = libros_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Genre']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    genres = []
    for entidad in entidades_consulta:

        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())

    candidate_movies = libros_dataset.copy()

     # Filtrar por género en 'categories' (solo si hay géneros extraídos)
    if len(genres) > 0:
        candidate_movies = candidate_movies[
            candidate_movies.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]


    # Mostrar el resultado final
    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)

    coseno = util.cos_sim(user_embedding, candidate_movies['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    # Extraer las recomendaciones
    recomendaciones = [candidate_movies['title'].iloc[i] for i in index]
    return recomendaciones

In [167]:
def busqueda_juegos(consulta):
    #convierto los valores de las columnas en listas
    juegos_dataset['directors'] = juegos_dataset['directors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    juegos_dataset['categories'] = juegos_dataset['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    consulta_usuario = consulta
    labels = ['Name','Genre','Year','Numplayers']
    entidades_consulta = modelo_gliner.predict_entities(traducir(consulta_usuario),labels,threshold=0.6)
    names = []
    years = []
    genres = []
    numplayers = 1
    for entidad in entidades_consulta:
        if entidad['label'] == 'Name':
            if entidad['text'].lower() == 'i':
                continue
            names.append(entidad['text'].lower())
        if entidad['label'] == 'Year':
            years.append(entidad['text'].lower())
        if entidad['label'] == 'Genre':
            genres.append(entidad['text'].lower())
        if entidad['label'] == 'Numplayers':
            numplayers = int(entidad['text'][0])

    candidate_movies = juegos_dataset.copy()
    # Filtrar por nombres en 'director' o 'actors'
    if len(names) > 0:
        candidate_movies = candidate_movies = juegos_dataset[
        juegos_dataset.apply(lambda row: any(name in [director for director in row['directors']] for name in names), axis=1)]
     # Filtrar por género en 'categories' (solo si hay géneros extraídos)
    if len(genres) > 0:
        candidate_movies = candidate_movies[
            candidate_movies.apply(lambda row: any(genre in [category for category in row['categories']] for genre in genres), axis=1)
        ]

    # Filtrar por años en 'years' (solo si hay años extraídos)
    if len(years) > 0:
        candidate_movies = candidate_movies[
            candidate_movies.apply(lambda row: any(str(year) in str(row['year']) for year in years), axis=1)
        ]
    #Filtrar por la cantidad de jugadores
    candidate_movies.apply(lambda row: row['minplayers'] <= numplayers <= row['maxplayers'], axis=1)
    # Mostrar el resultado final
    user_embedding = modelo_recomendaciones.encode(traducir(consulta_usuario),convert_to_tensor=True)

    coseno = util.cos_sim(user_embedding, candidate_movies['embeddings'].tolist())[0]
    index = coseno.argsort(descending=True)[:3].tolist()
    # Extraer las recomendaciones
    recomendaciones = [candidate_movies['title'].iloc[i] for i in index]
    return recomendaciones
  

In [160]:
print(busqueda_juegos('quiero un juego que se pueda jugar de a 3 personas y que sea de horror'))
print(busqueda_libros('Quiero leer un libro de misterio y fantasia'))
print(busqueda_peliculas('Quiero que la historia sea de accion y que actue Vin Diesel y sea del 2015'))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['Horrified', 'Zombicide Season 2: Prison Outbreak', 'Reign of Cthulhu']
['The Mysteries of Udolpho', 'Phantastes: A Faerie Romance for Men and Women', 'The Blue Fairy Book']
['Furious Seven', 'The Last Witch Hunter']
